In [5]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency


df = pd.read_csv('Green_Taxi_Trip_Data_2016.csv')


quasi_identifiers = ['Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count']
sensitive_attribute = 'Fare_amount'

t = 0.2

def chi_squared_distance(dist1, dist2):
    contingency_table = pd.crosstab(dist1, dist2)
    chi2, _, _, _ = chi2_contingency(contingency_table)
    return chi2

def satisfies_t_closeness(group_data, t, sensitive_attribute):
    unique_sensitive_values = group_data[sensitive_attribute].unique()
    
    if len(unique_sensitive_values) <= 1:
        return True  
    
    chi_squared = chi_squared_distance(group_data[sensitive_attribute], df[sensitive_attribute])
    
    return chi_squared <= t


grouped = df.groupby(quasi_identifiers)
t_closeness_satisfied_groups = [group for _, group in grouped if satisfies_t_closeness(group, t, sensitive_attribute)]
df_t_closeness_anonymized = pd.concat(t_closeness_satisfied_groups)
df_t_closeness_anonymized.to_csv('t_closeness_anonymized_dataset.csv', index=False)

MemoryError: Unable to allocate 500. MiB for an array with shape (4, 16385532) and data type float64